In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import polars as pl

In [ ]:
data = pd.read_csv("../data/ipeds_nsf.csv")

In [ ]:
# Read the data
nsf_ipeds = pl.read_csv("../data/ipeds_nsf.csv")
nsf_ipeds = nsf_ipeds.with_columns(pl.when(pl.col("Research Activity Designation")=="Research 1: Very High Research Spending and Doctorate Production").then(pl.lit("R1")).otherwise(pl.lit("R2")).alias("Research Activity Designation"))

id_uta = 228769
id_jhu = 162928
# aau = nsf_ipeds.filter(pl.col("AAU"))

nsf_avg = nsf_ipeds.group_by("UnitID").agg(
    pl.col("Institution Name").first(),
    pl.col("Year").first(),
    pl.col("State abbreviation").first(),
    pl.col("FIPS state code").first(),
    pl.col("Historically Black College or University").first(),
    pl.col("Land Grant Institution").first(),
    pl.col("Sector of institution").first(),
    pl.col("Degree of urbanization (Urban-centric locale)").first(),
    pl.col("Institution grants a medical degree").first(),
    pl.col("Carnegie Classification 2021: Basic").first(),
    pl.col("control").first(),
    pl.col("Institutional Classification").first(),
    pl.col("Student Access and Earnings Classification").first(),
    pl.col("Research Activity Designation").first(),
    pl.col("Award Level Focus").first(),
    pl.col("Academic Mix").first(),
    pl.col("Graduate Academic Program Mix").first(),
    pl.col("Size").first(),
    pl.col("Campus Setting").first(),
    pl.col("Highest Degree Awarded").first(),
    pl.col("Community Engagement").first(),
    pl.col("Leadership for Public Practice").first(),
    pl.col("AAU").first(),
    pl.col("Department of Energy").mean(),
    pl.col("National Aeronautics and Space Administration").mean(),
    pl.col("Other federal agency").mean(),
    pl.col("Nonfederal").mean(),
    pl.col("Department of Defense").mean(),
    pl.col("National Science Foundation").mean(),
    pl.col("Department of Agriculture").mean(),
    pl.col("Department of Health and Human Services").mean(),
)


In [ ]:
aau_data = nsf_avg.filter(pl.col("AAU")).with_columns(designation = pl.lit("AAU")).drop(["AAU", "Research Activity Designation"])
carn_data = nsf_avg.with_columns(designation = pl.col("Research Activity Designation")).drop(["AAU", "Research Activity Designation"])


In [ ]:
control_counts_aau = aau_data.group_by(["designation", "control"]).len().to_pandas()
control_counts_aau

In [ ]:
control_counts = carn_data.group_by(["designation", "control"]).len().to_pandas()
control_counts

In [ ]:
combined = pd.concat([control_counts, control_counts_aau], ignore_index=True)
combined

In [ ]:
pivot_df = combined.pivot(index=["designation"], columns="control", values="len").fillna(0).reset_index()
pivot_df

In [ ]:
pivot_df.columns

In [ ]:
# Melt to long format
df_melted = pivot_df.melt(
    id_vars="designation",
    var_name="Control",
    value_name="Count"
)
df_melted

In [ ]:
# Calculate proportions
df_melted["Proportion"] = df_melted.groupby("designation")["Count"].transform(lambda x: x / x.sum())


In [ ]:
# Plot the stacked bar chart
fig = px.bar(
    df_melted,
    x="designation",
    y="Proportion",
    color="Control",
    text="Count",
    title="Proportion of Public vs Private Institutions",
    width=1000,
)

fig.update_layout(barmode="stack", yaxis_tickformat=".0%")
fig.show()

In [ ]:
size_counts = carn_data.group_by(["designation", "Size"]).len().to_pandas()
size_counts_aau = aau_data.group_by(["designation", "Size"]).len().to_pandas()

In [ ]:
combined = pd.concat([size_counts, size_counts_aau], ignore_index=True)

In [ ]:
size_order = ["Very Small", "Small", "Medium", "Large", "Very Large"]
combined["Size"] = pd.Categorical(combined["Size"], categories=size_order, ordered=True)
combined

In [ ]:
# Sort by Size
combined_sorted = combined.sort_values("Size")
combined_sorted

In [ ]:
combined_sorted["Proportions"] = combined_sorted.groupby("designation")["len"].transform(lambda x: x / x.sum())
combined_sorted

In [ ]:
# Plot the stacked bar chart
fig = px.bar(
    combined_sorted,
    x="designation",
    y="Proportions",
    color="Size",
    text="len",
    title="Size Proportions",
    width=1000,
)

fig.update_layout(barmode="stack", yaxis_tickformat=".0%")
fig.show()

In [ ]:
medical_counts = carn_data.group_by(["designation", "Institution grants a medical degree"]).len().to_pandas()
medical_counts_aau = aau_data.group_by(["designation", "Institution grants a medical degree"]).len().to_pandas()

In [ ]:
combined = pd.concat([medical_counts, medical_counts_aau], ignore_index=True)
combined

In [ ]:
combined["Proportions"] = combined.groupby("designation")["len"].transform(lambda x: x / x.sum())
combined

In [ ]:
combined["Institution grants a medical degree"] = combined["Institution grants a medical degree"].map({1: "Yes", 2: "No"})

In [ ]:
combined

In [ ]:
# Plot the stacked bar chart
fig = px.bar(
    combined,
    x="designation",
    y="Proportions",
    color="Institution grants a medical degree",
    text="len",
    title="Institution grants a medical degree",
    width=1000,
)

fig.update_layout(barmode="stack", yaxis_tickformat=".0%")
fig.show()

In [ ]:
funding_columns = [
    "Department of Energy",
    "National Aeronautics and Space Administration",
    "Department of Defense",
    "National Science Foundation",
    "Department of Health and Human Services",
    "Other federal agency",
    "Department of Agriculture",
    "Nonfederal",
]

# Map full names to abbreviations
abbreviated_labels = [
    "DOE", "NASA", "DOD", "NSF", "HHS",   "Other Fed", "USDA","Nonfed",
]

In [ ]:
aau_data_avg = aau_data.to_pandas()[funding_columns].mean()
r1_data_avg = carn_data.filter(pl.col("designation") == "R1").to_pandas()[funding_columns].mean()
r2_data_avg = carn_data.filter(pl.col("designation") == "R2").to_pandas()[funding_columns].mean()

In [ ]:
r2_data_avg

In [ ]:
aau_means = aau_data_avg*1000
r1_means = r1_data_avg*1000
r2_means = r2_data_avg*1000

In [ ]:
# Create comparison bar chart
fig = go.Figure()

fig.add_trace(go.Bar(
    x=abbreviated_labels,
    y=aau_means,
    name="AAU Average",
    marker_color="royalblue",
))

fig.add_trace(go.Bar(
    x=abbreviated_labels,
    y=r1_means,
    name="R1 Average",
    marker_color="seagreen",
))

fig.add_trace(go.Bar(
    x=abbreviated_labels,
    y=r2_means,
    name="R2 Average",
    marker_color="red",
))

fig.update_layout(
    title="Comparison of Average Research Funding",
    xaxis_title="Funding Source",
    yaxis_title="Average Funding Amount ($)",
    barmode="group",
    xaxis_tickangle=-45,
    height=600,
    width = 1000,
)

fig.show()

In [ ]:
funding_columns = [
    "Department of Energy",
    "National Aeronautics and Space Administration",
    "Other federal agency",
    "Department of Defense",
    "National Science Foundation",
    "Department of Health and Human Services",
    "Department of Agriculture",
    "Nonfederal",
]

In [ ]:
# Calculate total funding metrics
total_funding_cols_w_doa = funding_columns[:-1]
total_funding_cols_wo_doa = funding_columns[:-2]

In [ ]:
df = carn_data.filter(pl.col("designation") == "R1").to_pandas()
# Compute total columns
df["Total (with DOA)"] = df[total_funding_cols_w_doa].sum(axis=1)
df["Total (without DOA)"] = df[total_funding_cols_wo_doa].sum(axis=1)

# Add total columns to the list for analysis
all_metrics = [*funding_columns, "Total (with DOA)", "Total (without DOA)"]

In [ ]:
df[df["Institution Name"]=="The University of Texas at El Paso"][all_metrics]

In [ ]:
categorical_columns = [
    "Historically Black College or University", "Land Grant Institution",
    "Sector of institution", "Degree of urbanization (Urban-centric locale)",
    "Institution grants a medical degree", "control", "Institutional Classification",
    "Student Access and Earnings Classification",
    "Award Level Focus", "Academic Mix", "Graduate Academic Program Mix",
    "Size", "Campus Setting", "Highest Degree Awarded",
]

In [ ]:
df["designation"].head()

In [ ]:
# Create a pipeline for preprocessing
# Process categorical data
from sklearn.discriminant_analysis import StandardScaler


cat_features = [col for col in categorical_columns if col in df.columns]
# Handle missing values in categorical features if any
for col in cat_features:
    df[col] = df[col].fillna("Unknown")

# One-hot encode categorical features
encoded_cats = pd.get_dummies(df[cat_features], drop_first=False)


# Scale numerical features
scaler = StandardScaler()
scaled_nums = pd.DataFrame(
    scaler.fit_transform(df[all_metrics]),
    columns=all_metrics,
    index=df.index,
)

# Combine processed features
processed_df = pd.concat([scaled_nums, encoded_cats], axis=1)

In [ ]:
processed_df.head()

In [ ]:
processed_df.columns

In [ ]:
# Perform K-means clustering
# Determine optimal number of clusters using the elbow method
from sklearn.cluster import KMeans


inertia_values = []
k_range = range(2, min(10, len(processed_df) - 1))
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(processed_df)
    inertia_values.append(kmeans.inertia_)

# Plot elbow method
fig_elbow = px.line(
    x=list(k_range), y=inertia_values,
    title="Elbow Method for Optimal k",
    labels={"x": "Number of Clusters (k)", "y": "Inertia"},
)
fig_elbow.add_shape(
    type="line",
    line={"dash": "dash", "color": "gray"},
    x0=4, y0=min(inertia_values), x1=4, y1=max(inertia_values),
)


In [ ]:
# Choose k based on elbow method (for now, let's assume k=4)
from sklearn.decomposition import PCA


k_optimal = 3
kmeans = KMeans(n_clusters=k_optimal, random_state=42)
df["Cluster"] = kmeans.fit_predict(processed_df)

# Add cluster information to original data
uta_cluster = df.loc[df["UnitID"] == id_uta, "Cluster"].values[0]

# Perform PCA for visualization
pca = PCA(n_components=2)  # noqa: F821
pca_result = pca.fit_transform(processed_df)
df["PCA1"] = pca_result[:, 0]
df["PCA2"] = pca_result[:, 1]

# Create scatter plot with PCA
fig_pca = px.scatter(
    df, x="PCA1", y="PCA2", color="Cluster",
    hover_data=["Institution Name"],
    title="University Clusters based on PCA",
    labels={"PCA1": "Principal Component 1", "PCA2": "Principal Component 2"},
    color_continuous_scale=px.colors.qualitative.Set1,
)

# Highlight UTA in the plot if it exists
uta_point = df[df["UnitID"] == id_uta]
fig_pca.add_trace(
    go.Scatter(
        x=uta_point["PCA1"],
        y=uta_point["PCA2"],
        mode="markers",
        marker={
            "color": "black",
            "size": 15,
            "line": {"width": 2, "color": "black"},
        },
        name="UTA",
        hoverinfo="text",
        text=uta_point["Institution Name"],
    ),
)

fig_pca.show()


In [ ]:
cluster_summary = df.groupby("Cluster")[categorical_columns].agg(lambda x: x.value_counts().index[0])
print(cluster_summary.loc[uta_cluster])

In [ ]:
cluster_summary = df.groupby("Cluster")[all_metrics].agg(lambda x: x.mean())
print(cluster_summary.loc[uta_cluster])

In [ ]:
df[df["Institution Name"]=="The University of Texas at El Paso"][all_metrics]